<a href="https://colab.research.google.com/github/ZS520L/AUSTAI/blob/main/freegpt4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import requests
import base64
from tqdm import tqdm

def search_fofa(query, email, key, page):
    """
    :param query: 搜索查询。
    :param email: FOFA账户邮箱。
    :param key: FOFA API密钥。
    :return: 搜索结果。
    """
    base_url = 'https://fofa.info/api/v1/search/all?'
    query_enc = base64.b64encode(query.encode()).decode()
    params = {
        'email': email,
        'key': key,
        'qbase64': query_enc,
        'size': 10000,
        'page': page
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def test_models(url, models):
    headers = {
        "Content-Type": "application/json"
    }

    available_models = []
    for model in models:
        try:
            data = {'model': model, 'messages': [{'role':'user', 'content':'苏轼为什么暴打苏东坡？'}], 'stream': True}
            response = requests.post(url + '/api/openai' + '/v1/chat/completions', headers=headers, data=json.dumps(data), stream=True)
            if response.status_code == 200 and ("chat.completion.chunk" in response.text):
                # print(response.text,'\n')
                available_models.append(model)
        except:
            break
    return available_models + ['gpt4']

models = ['gpt-3.5-turbo', 'gpt-3.5-turbo-16k', 'gpt-3.5-turbo-1106', 'gpt-4-0613', 'gpt-4-32k', 'gpt-4-1106-preview', 'gpt-4-vision-preview']


# 使用示例
# email = 'li1u888@163.com'
# key = '5dee7d8a386a3ac07079b229fc80142e'

email = 'dffjka@open_wechat'
key = 'ed28fbd30452c5154aa8e6a5d4232592'
query = '"chat" && icon_hash="1296353639" && country="JP"'
page = 1

results = search_fofa(query, email, key, page)
try:
  print('成功检索到',len(results['results']),'条数据！')
except:
  print(results)

成功检索到 2219 条数据！


In [ ]:
import json

def extract_content(data_string):
    lines = data_string.split("\n")  # split by newline
    content = ""

    for line in lines:
        if line.startswith('data:'):
            try:
                data_str = line[6:]  # remove 'data: '
                data_json = json.loads(data_str)
                choices = data_json.get('choices', [])
                for choice in choices:
                    delta = choice.get('delta', {})
                    content += delta.get('content', '')
            except json.JSONDecodeError:
                continue

    return content

headers = {
    "Content-Type": "application/json"
}

data = {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'user', 'content': '苏轼为什么暴打苏东坡？'}],'stream':True}

if results:
    for result in tqdm(results['results']):
        link = result[0]
        try:
            # 替换可变部分的链接
            url = link + '/api/openai/v1/chat/completions'
            response = requests.post(url, headers=headers, data=json.dumps(data), stream=True, timeout=1)
            # 如果状态码为200，表示请求成功
            if response.status_code == 200 and ("chat.completion.chunk" in response.text):
                print('###'*10)
                print(extract_content(response.text))
                print('接口地址：', link+'/api/openai')
                print("可用模型：", test_models(link, models))
                with open("output.txt", "a") as f:  # "a"模式表示追加模式，如果文件不存在，将会创建
                    f.write('接口地址：' + link + '/api/openai' + '\n')
                    f.write("可用模型：" + str(test_models(link, models)) + '\n')
                print('###'*10)
            else:
                # print(f"Request to {url} failed with status code {response.status_code}")
                pass
        except Exception as e:
            # print(f"An error occurred while making a request to {url}: {e}")
            pass
else:
    print("无法获取数据或没有找到匹配结果。")

 28%|██▊       | 623/2219 [01:54<06:35,  4.04it/s]

##############################
èè½¼å¹¶æ²¡ææ´æèä¸å¡ãèä¸å¡ï¼å­å­ç»ï¼åèè½¼ï¼å­å­ç»ï¼æ¯åä¸äººï¼æ¯åå®æ¶æçæå­¦å®¶ãæ¿æ²»å®¶ãä¹¦æ³å®¶ãç¾é£å®¶ç­å¤æ¹é¢æååºä¼çäººç©ãèè½¼åèä¸å¡æ¯åä¸äººçä¸ååå­ï¼ä¸»è¦æ¯å ä¸ºå¶å­å·çååä½¿å¾äººä»¬å¨ä¸ååºåä½¿ç¨ä¸åçåå­ã
接口地址： https://xiutie.chat.dplus.tech/api/openai
可用模型： ['gpt-3.5-turbo', 'gpt-3.5-turbo-16k', 'gpt-3.5-turbo-1106', 'gpt4']


 28%|██▊       | 625/2219 [03:05<4:54:05, 11.07s/it]

##############################


 71%|███████   | 1570/2219 [05:49<03:46,  2.87it/s]